In [1]:
import pandas as pd
from tqdm.auto import tqdm

In [2]:
df = pd.read_csv('delivery_orders_march.csv')

In [3]:
df.buyeraddress = df.buyeraddress.str.split().str[-1].str.capitalize()
df.selleraddress = df.selleraddress.str.split().str[-1].str.capitalize()

In [4]:
df.buyeraddress.value_counts()

Manila      1560698
Luzon        804888
Visayas      479595
Mindanao     331132
Name: buyeraddress, dtype: int64

In [5]:
df.selleraddress.value_counts()

Manila    3176312
Luzon           1
Name: selleraddress, dtype: int64

In [6]:
for col in tqdm(['pick', '1st_deliver_attempt', '2nd_deliver_attempt']):
    df[col] = (pd.to_datetime(df[col], unit='s') + pd.DateOffset(hours=8)).dt.date

100%|██████████| 3/3 [01:13<00:00, 24.46s/it]


In [7]:
matrix = {'Luzon': 5, 'Manila': 3, 'Mindanao': 7, 'Visayas': 7}

In [8]:
df['sla'] = df.apply(lambda x: matrix[x.buyeraddress], axis=1)

In [9]:
df

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,sla
0,2215676524,2020-03-02,2020-03-05,NaT,Manila,Manila,3
1,2219624609,2020-03-04,2020-03-06,2020-03-10,Manila,Manila,3
2,2220979489,2020-03-04,2020-03-06,NaT,Manila,Manila,3
3,2221066352,2020-03-05,2020-03-07,NaT,Manila,Manila,3
4,2222478803,2020-03-04,2020-03-06,NaT,Luzon,Manila,5
...,...,...,...,...,...,...,...
3176308,31504087640510,2020-04-02,2020-04-04,NaT,Manila,Manila,3
3176309,31504147352227,2020-04-03,2020-04-06,2020-04-07,Manila,Manila,3
3176310,31504462290482,2020-04-02,2020-04-04,NaT,Manila,Manila,3
3176311,31504851495943,2020-04-03,2020-04-07,NaT,Luzon,Manila,5


In [10]:
holidays = pd.to_datetime(['2020-03-08', '2020-03-15', '2020-03-22', '2020-03-25', '2020-03-29', '2020-03-30', '2020-03-31', '2020-04-05'])

def working_days(first, second):
    hol = 0
    for i in holidays:
        if first <= i <= second:
            hol += 1
    return (second - first).days - hol

def solve(row):
    orderid, pick, first, second, buyer, seller, sla = row
    if (first - pick).days > sla + 5:
        return 1
    if (first - pick).days > 3 and working_days(pick, first) > sla:
        return 1
    if pd.isnull(second):
        return 0
    if (second - first).days <= 3:
        return 0
    if working_days(first, second) > 3:
        return 1
    return 0

solve(df.iloc[0])

0

In [11]:
df['is_late'] = df.apply(solve, axis=1)

In [12]:
out = df[['orderid', 'is_late']]
out.to_csv('submission.csv', index=False)

In [13]:
out.is_late.value_counts()

0    2413891
1     762422
Name: is_late, dtype: int64